In [3]:
!pip install -U spacy
!pip install spacy_transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Import required libraries and install any necessary packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import pandas as pd

In [28]:
ukr_data = pd.read_csv('./data/uk_geo_dataset.csv', nrows=200000, converters={"loc_markers": eval, "org_markers": eval, "per_markers": eval})

ukr_data

,text,loc_markers,org_markers,per_markers,is_valid
0,"Чим довше мають скачки тиску гіпертензією, тим...",[],[],[],0
1,"А поки що починали цвісти троянди, випускники ...",[],[],"[(114, 131)]",0
2,"Крім того, в 2020 р. багато експертів прогнозу...",[],[],[],0
3,"Сильно сумніваюся, що ви зупините свій вибір н...",[],[],[],0
4,Цей унікальний правовий важіль утворено 1998 р...,[],[],[],0
...,...,...,...,...,...
199995,"""Ніхто навіть не здивований, у світлі останніх...",[],[],[],0
199996,"Бувало, жив одразу з двома жінками, пропонуючи...",[],[],[],0
199997,Продукти маркували товарними знаками відомих в...,[],[],[],0
199998,Онук британської королеви з’явився в етері бри...,[],[],[],0


In [29]:
rus_data = pd.read_csv('./data/ru_geo_dataset.csv', nrows=200000, converters={"loc_markers": eval, "org_markers": eval, "per_markers": eval})

rus_data

,text,loc_markers,org_markers,per_markers,doc_id,sent_id
0,Вице-премьер по социальным вопросам Татьяна Го...,"[(82, 88)]","[(149, 160)]","[(36, 52)]",0,0
1,"По словам Голиковой, чаще всего онкологические...","[(89, 98), (100, 108), (110, 118), (121, 139),...",[],"[(10, 19)]",0,1
2,"Вице-премьер напомнила, что главные факторы см...","[(57, 63)]",[],[],0,2
3,"В начале года стало известно, что смертность о...",[],[],[],0,3
4,"По данным Росстата, в 2017 году от рака умерли...",[],"[(10, 18)]",[],0,4
...,...,...,...,...,...,...
199995,"Это было очень бестактно, — говорит она.",[],[],[],16979,9
199996,— Когда мне предложили поучаствовать в съемках...,[],[],[],16979,10
199997,"По словам девушки, она надеется, что ее фото п...",[],[],[],16979,11
199998,"«Было здорово увидеть, что люди не показывают ...",[],[],[],16979,12


In [30]:
ukr_rus_data = pd.concat([ukr_data, rus_data]).reset_index(drop=True)

In [31]:
ukr_rus_data

,text,loc_markers,org_markers,per_markers,is_valid,doc_id,sent_id
0,"Чим довше мають скачки тиску гіпертензією, тим...",[],[],[],0.0,NaN,NaN
1,"А поки що починали цвісти троянди, випускники ...",[],[],"[(114, 131)]",0.0,NaN,NaN
2,"Крім того, в 2020 р. багато експертів прогнозу...",[],[],[],0.0,NaN,NaN
3,"Сильно сумніваюся, що ви зупините свій вибір н...",[],[],[],0.0,NaN,NaN
4,Цей унікальний правовий важіль утворено 1998 р...,[],[],[],0.0,NaN,NaN
...,...,...,...,...,...,...,...
399995,"Это было очень бестактно, — говорит она.",[],[],[],NaN,16979.0,9.0
399996,— Когда мне предложили поучаствовать в съемках...,[],[],[],NaN,16979.0,10.0
399997,"По словам девушки, она надеется, что ее фото п...",[],[],[],NaN,16979.0,11.0
399998,"«Было здорово увидеть, что люди не показывают ...",[],[],[],NaN,16979.0,12.0


In [32]:
train_data = []

for _, row in ukr_rus_data.iterrows():
    entities = {'entities': []}

    for loc in row['loc_markers']:
      entities['entities'].append((loc[0], loc[1], 'Location'))

    for org in row['org_markers']:
      entities['entities'].append((org[0], org[1], 'Organization'))

    for per in row['per_markers']:
      entities['entities'].append((per[0], per[1], 'Person'))

    annotation = (row['text'], entities)

    train_data.append(annotation)

In [33]:
len(train_data)

400000

In [39]:
!python3 -m spacy init fill-config ./config/base_config_3.cfg ./config/config.cfg

✔ Auto-filled config with all values
✔ Saved config
config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [40]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [41]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_data, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

(320000, 80000)

In [42]:
# Open a file to log errors during annotation processing
file = open('./trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('./trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('./trained_models/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 80000/80000 [00:13<00:00, 5839.51it/s]


In [43]:
 !python3 -m spacy train ./config/config.cfg  --output ./trained_models/output  --paths.train ./trained_models/train  --paths.dev ./trained_models/test_data.spacy


ℹ Saving to output directory: trained_models/output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     39.67    0.08    0.13    0.05    0.00
  0     200          6.19   1111.72   35.46   46.97   28.48    0.35
  0     400         15.92    679.19   39.65   59.57   29.72    0.40
  0     600         23.43    641.97   36.17   88.13   22.76    0.36
  0     800        454.81    779.33   37.40   88.80   23.68    0.37
  0    1000        125.55    814.05   42.75   77.05   29.58    0.43
  0    1200         46.16    819.80   42.58   81.01   28.88    0.43
  0    1400         78.54    985.71

In [ ]:
nlp = spacy.load('./trained_models/output/model-best')

In [ ]:
test_data = pd.read_csv('./data/test.csv')

test_data

,text_id,text,locations
0,0,"❗️Кількість поранених зросла до трьох, – Кличк...",[]
1,1,"🥤В Києві за 91,13 млн гривень починаються робо...",[]
2,2,▪️Сьогодні вночі росіяни завдали ракетного уда...,[]
3,3,Наразі у запасах росіян найбільше балістичних ...,[]
4,4,"⛸В один день, 29 серпня, ДП ""Центральна учбово...",[]
...,...,...,...
472,472,Допомога на відновлення: п’ять громад Київщини...,[]
473,473,Пророцтво Стругацьких: незвичайна бактерія Бра...,[]
474,474,Із 1 вересня УГКЦ і ПЦУ перейшли на новий церк...,[]
475,475,🦇 У Києві з Дніпра виловили іноземця в масці Б...,[]


In [ ]:
for _, row in test_data.iterrows():
  doc = nlp(row['text'])

  for ent in doc.ents:
    # Print the recognized text and its corresponding label
    print(ent.text, "  ->>>>  ", ent.label_)

Київ   ->>>>   Location
Києві   ->>>>   Location
Гоголеве   ->>>>   Location
Миргородського району   ->>>>   Location
Полтавської області   ->>>>   Location
проспекті Академіка   ->>>>   Location
Брянськ   ->>>>   Location
Забуччя   ->>>>   Location
Київ   ->>>>   Location
Солом’янському районі столиці   ->>>>   Location
вулиці Новій   ->>>>   Location
Солом’янському районі   ->>>>   Location
Левка Лукʼяненка   ->>>>   Location
Київ   ->>>>   Location
Київ   ->>>>   Location
Києві   ->>>>   Location
Києві   ->>>>   Location
Житомирщині   ->>>>   Location
Л-39   ->>>>   Location
РФ   ->>>>   Location
Києві   ->>>>   Location
Дарницькому районі столиці

Попередньо   ->>>>   Location
Дарницькому районі   ->>>>   Location
Лісовий проспект   ->>>>   Location
Києві   ->>>>   Location
Солом’янському районі   ->>>>   Location
Україні   ->>>>   Location
Україна   ->>>>   Location
Харківщини   ->>>>   Location
Шевченківському районі   ->>>>   Location
вул. Димитрівська   ->>>>   Location
Криму  